In [255]:
import torch
import sentencepiece
from transformers import AutoTokenizer, AutoModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

ValueError: Couldn't instantiate the backend tokenizer from one of: (1) a `tokenizers` library serialization file, (2) a slow tokenizer instance to convert or (3) an equivalent slow tokenizer class to instantiate and convert. You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [241]:
import os, csv

data = []
sentences = []
current_sentence = []
length = 0

out_of_vocab = {'ड़': 'ड़', 'ढ़': 'ढ़', '‘': '', '‘': '', "'": '', '"': ''}

for entry in os.scandir('../annotations/lp_aryaman'):
    if entry.path.endswith('.csv'):
        
        with open(entry, 'r') as fin:
            reader = csv.reader(fin)
            for i, row in enumerate(reader):
                if i == 0: continue
                
                for i, j in out_of_vocab.items():
                    row[0] = row[0].replace(i, j)
                
                if row[0] == '':
                    if current_sentence:
                        sentences.append(current_sentence)
                        length = 0
                    current_sentence = []
                else:
                    if row[2]: data.append([row[0], row[2], row[3], len(sentences), length])
                    length += len(row[0])
                    current_sentence.append(row[0])

        if current_sentence:
            sentences.append(current_sentence)
            
# this is our training data
print(data[:10])
print(len(data))

[['मैंने', 'Experiencer', 'Agent', 2, 0], ['तेरा', 'Theme', 'Theme', 2, 5], ['उसकी', 'Gestalt', 'Gestalt', 2, 14], ['तक', 'Duration', 'EndTime', 3, 9], ['मेरे', 'Possessor', 'Possessor', 3, 11], ['का', 'Purpose', 'Purpose', 3, 26], ['का', 'Gestalt', 'Gestalt', 3, 51], ['इसका', 'Topic', 'Topic', 4, 0], ['मुझे', 'Recipient', 'Recipient', 4, 7], ['तुझसे', 'Agent', 'Ancillary', 4, 11]]
2450


In [242]:
# tokenize sentences
print(' '.join(sentences[0]))
sentences = [tokenizer.tokenize('[CLS] ' + ' '.join(sentence) + ' [SEP]') for sentence in sentences]

print(len(sentences))
print(sentences[:5])

ओह !
1722
[['[CLS]', 'ओ', '##ह', '!', '[SEP]'], ['[CLS]', 'न', '##न्', '##हे', '##ं', 'र', '##ाज', '##क', '##ु', '##मा', '##र', '!', '[SEP]'], ['[CLS]', 'म', '##ैं', '##ने', 'ते', '##रा', 'जीवन', ',', 'उसकी', 'उ', '##दा', '##सी', 'ध', '##ीर', '##े', '-', 'ध', '##ीर', '##े', 'स', '##म', '##झ', 'ली', 'थी', '[SEP]'], ['[CLS]', 'बहुत', 'दिन', '##ों', 'तक', 'मे', '##रे', 'पास', 'म', '##न', 'ब', '##ह', '##ला', '##ने', 'का', 'एक', 'मात्रा', 'स', '##ा', '##धन', 'था', 'स', '##ूर', '##्यास', '##्त', 'का', 'स', '##ौ', '##ंदर', '##्य', '।', '[SEP]'], ['[CLS]', 'इसका', 'पता', 'म', '##ु', '##झ', '##े', 'त', '##ु', '##झ', '##से', 'म', '##ुल', '##ा', '##का', '##त', 'के', 'च', '##ौ', '##थे', 'दिन', 'च', '##ला', 'जब', 'त', '##ून', '##े', 'म', '##ु', '##झ', '##से', 'कहा', ':', '[SEP]']]


In [243]:
# Map the token strings to their vocabulary indeces.
indexed_tokens = [tokenizer.convert_tokens_to_ids(sentence) for sentence in sentences]
print(indexed_tokens[0])
print(sentences[0])


[101, 863, 17110, 106, 102]
['[CLS]', 'ओ', '##ह', '!', '[SEP]']


In [231]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [244]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers.
outputs = []
with torch.no_grad():
    
    for i, sentence in enumerate(indexed_tokens):
        if len(sentence) == 0:
            outputs.append([])
            continue
        # Mark each of the 22 tokens as belonging to sentence "1".
        segments_ids = [1] * len(sentence)
        if i % 10 == 0: print(i)
        
        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([sentence])
        segments_tensors = torch.tensor([segments_ids])
        
        outputs.append(model(tokens_tensor, segments_tensors))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720


In [221]:
outputs[0]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3722, -0.2836,  0.2937,  ...,  0.6907,  0.2142,  0.2329],
         [ 0.3534, -0.5321,  0.2235,  ...,  1.0148,  0.1673,  0.2986],
         [ 0.6164, -0.2585,  0.7959,  ...,  0.7183, -0.1231, -0.1010],
         [ 0.3348, -0.2729,  0.5100,  ...,  0.6602,  0.1632,  0.2698],
         [ 0.4161, -0.2375,  0.4754,  ...,  0.4744,  0.0905,  0.2347]]]), pooler_output=tensor([[ 3.3972e-01,  2.4252e-01,  8.5810e-02,  6.2661e-02, -1.4015e-01,
          1.4702e-01,  7.4292e-02, -2.9657e-02,  6.9910e-02,  1.9596e-01,
         -1.1321e-03, -1.0474e-01, -3.1804e-01, -1.2298e-01,  1.4344e-01,
          9.6834e-02,  3.1660e-02,  1.9304e-02, -1.1957e-01, -1.4818e-01,
         -9.7831e-01, -5.8960e-02,  1.7460e-01, -2.0389e-01, -1.4001e-01,
         -2.4887e-01, -1.1476e-01, -1.5955e-01,  2.1249e-01, -1.5593e-01,
         -1.2544e-01, -9.8624e-01,  3.2594e-01,  4.3030e-01,  3.9739e-01,
         -7.1544e-02,  5.5542e-02,  2.3614e-01,

In [245]:
all_embeddings = []
for i, output in enumerate(outputs):
    if i % 100 == 0: print(i)
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(output[2], dim=0)
    
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    all_embeddings.append(token_embeddings)

print(len(sentences), len(all_embeddings))
for i in range(len(sentences)):
    if len(sentences[i]) != len(all_embeddings[i]):
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1722 1722


In [247]:
cur_word_length = 0
concatenated_embeddings = []
for i, token_embeddings in enumerate(all_embeddings):
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = {}
    print(i)

    # `token_embeddings` is a [22 x 12 x 768] tensor.
    # print(token_embeddings.size())

    # For each token in the sentence...
    length = 0
    for j, token in enumerate(token_embeddings):

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
        
        # Sum the vectors from the last four layers.
        # sum_vec = torch.sum(token[-4:], dim=0)

        # Use `cat_vec` to represent `token`.
        if j == 0:
            token_vecs_cat[length] = [cat_vec, sentences[i][j]]
        elif sentences[i][j].startswith('##'):
            length += len(sentences[i][j]) - 2
        else:
            token_vecs_cat[length] = [cat_vec, sentences[i][j]]
            length += len(sentences[i][j])
    
    concatenated_embeddings.append(token_vecs_cat)

#     print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [248]:
for _, i in enumerate(data):
    if i[4] not in concatenated_embeddings[i[3]]:
        print(sentences[i[3]])
        print([i for i in concatenated_embeddings[i[3]]])
        print(i)

['[CLS]', 'म', '##ैं', 'ते', '##री', 'भ', '##ेड', '##़', 'का', 'म', '##ु', '##ं', '##ह', 'ब', '##ंद', 'करने', 'के', 'लिए', 'एक', 'जा', '##बा', '[UNK]', 'बना', 'द', '##ू', '##ंगा', ',', 'ते', '##रे', 'फ', '##ूल', 'के', 'लिए', 'क', '##व', '##च', 'बना', 'द', '##ू', '##ंगा', '।', '[SEP]']
[0, 3, 7, 11, 13, 17, 20, 24, 26, 29, 31, 35, 40, 43, 48, 49, 53, 56, 58, 61, 64, 67, 72, 73]
['तेरे', 'Possessor', 'Possessor', 96, 45]
['[CLS]', 'म', '##ैं', 'ते', '##री', 'भ', '##ेड', '##़', 'का', 'म', '##ु', '##ं', '##ह', 'ब', '##ंद', 'करने', 'के', 'लिए', 'एक', 'जा', '##बा', '[UNK]', 'बना', 'द', '##ू', '##ंगा', ',', 'ते', '##रे', 'फ', '##ूल', 'के', 'लिए', 'क', '##व', '##च', 'बना', 'द', '##ू', '##ंगा', '।', '[SEP]']
[0, 3, 7, 11, 13, 17, 20, 24, 26, 29, 31, 35, 40, 43, 48, 49, 53, 56, 58, 61, 64, 67, 72, 73]
['के', 'Beneficiary', 'Beneficiary', 96, 52]
['[CLS]', 'इसी', 'तरह', 'र', '##ाज', '##क', '##ु', '##मा', '##र', 'से', 'म', '##ुल', '##ा', '##का', '##त', 'के', 'ती', '##सर', '##े', 'दिन', 'म', '##ु',

In [154]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
same_bank = 1 - cosine(concatenated_embeddings[14][6][0], concatenated_embeddings[14][11][0])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
diff_bank = 1 - cosine(concatenated_embeddings[14][6][0], concatenated_embeddings[14][3][0])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

diff_bank = 1 - cosine(concatenated_embeddings[4][11][0], concatenated_embeddings[10][1][0])
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.91
Vector similarity for *different* meanings:  0.82
Vector similarity for *different* meanings:  0.78


In [159]:
print(len(data), len(concatenated_embeddings))
print(data[0])
for i in data:
    print(sentences[i[3]], i)
    if len(concatenated_embeddings[i[3]][i[4]]) != 2:
        print(i)

2450 1562
['मैंने', 'Experiencer', 'Agent', 2, 1]
['[CLS]', 'म', '##ैं', '##ने', 'ते', '##रा', 'जीवन', ',', 'उसकी', 'उ', '##दा', '##सी', 'ध', '##ीर', '##े', '-', 'ध', '##ीर', '##े', 'स', '##म', '##झ', 'ली', 'थी', '[SEP]'] ['मैंने', 'Experiencer', 'Agent', 2, 1]
['[CLS]', 'म', '##ैं', '##ने', 'ते', '##रा', 'जीवन', ',', 'उसकी', 'उ', '##दा', '##सी', 'ध', '##ीर', '##े', '-', 'ध', '##ीर', '##े', 'स', '##म', '##झ', 'ली', 'थी', '[SEP]'] ['तेरा', 'Theme', 'Theme', 2, 2]
['[CLS]', 'म', '##ैं', '##ने', 'ते', '##रा', 'जीवन', ',', 'उसकी', 'उ', '##दा', '##सी', 'ध', '##ीर', '##े', '-', 'ध', '##ीर', '##े', 'स', '##म', '##झ', 'ली', 'थी', '[SEP]'] ['उसकी', 'Gestalt', 'Gestalt', 2, 5]
['[CLS]', 'बहुत', 'दिन', '##ों', 'तक', 'मे', '##रे', 'पास', 'म', '##न', 'ब', '##ह', '##ला', '##ने', 'का', 'एक', 'मात्रा', 'स', '##ा', '##धन', 'था', 'स', '##ूर', '##्यास', '##्त', 'का', 'स', '##ौ', '##ंदर', '##्य', '।', '[SEP]'] ['तक', 'Duration', 'EndTime', 3, 3]
['[CLS]', 'बहुत', 'दिन', '##ों', 'तक', 'मे', '##रे', 'पास', 

['[CLS]', 'र', '##ाज', '##क', '##ु', '##मा', '##र', 'ग', '##ुल', '##ा', '##ब', '##ों', 'के', 'पास', 'गया', ',', '"', 'त', '##ु', '##म', 'लोग', 'मे', '##रे', 'ग', '##ुल', '##ा', '##ब', 'जैसे', 'ब', '##िल', '##्क', '##ुल', 'नहीं', 'हो', ',', '"', 'वह', 'ब', '##ोल', '##ा', ',', '"', 'त', '##ु', '##म', 'सब', 'न', '##ग', '##ण', '##्य', 'हो', 'किसी', 'ने', 'त', '##ु', '##म्', '##हे', '##ं', 'अपना', '##या', 'नहीं', '।', '[SEP]'] ['जैसे', 'ComparisonRef', 'ComparisonRef', 522, 12]
['[CLS]', 'र', '##ाज', '##क', '##ु', '##मा', '##र', 'ग', '##ुल', '##ा', '##ब', '##ों', 'के', 'पास', 'गया', ',', '"', 'त', '##ु', '##म', 'लोग', 'मे', '##रे', 'ग', '##ुल', '##ा', '##ब', 'जैसे', 'ब', '##िल', '##्क', '##ुल', 'नहीं', 'हो', ',', '"', 'वह', 'ब', '##ोल', '##ा', ',', '"', 'त', '##ु', '##म', 'सब', 'न', '##ग', '##ण', '##्य', 'हो', 'किसी', 'ने', 'त', '##ु', '##म्', '##हे', '##ं', 'अपना', '##या', 'नहीं', '।', '[SEP]'] ['ने', 'Agent', 'Agent', 522, 27]
['[CLS]', 'र', '##ाज', '##क', '##ु', '##मा', '##र', 'ग', '##ुल

['[CLS]', '"', 'यह', 'मे', '##रा', 'काम', 'है', '।', '"', '[SEP]'] ['मेरा', 'Gestalt', 'Gestalt', 1072, 3]
['[CLS]', 'र', '##ाज', '##क', '##ु', '##मा', '##र', 'ने', 'कहा', '।', '[SEP]'] ['ने', 'Originator', 'Agent', 1075, 2]
['[CLS]', '"', 'इसमें', 'स', '##म', '##झ', '##ने', 'की', 'कोई', 'बात', 'नहीं', '।', '[SEP]'] ['इसमें', 'Topic', 'Topic', 1076, 2]
['[CLS]', '"', 'इसमें', 'स', '##म', '##झ', '##ने', 'की', 'कोई', 'बात', 'नहीं', '।', '[SEP]'] ['की', 'Purpose', 'Purpose', 1076, 4]
['[CLS]', 'काम', 'का', 'म', '##त', '##ल', '##ब', 'काम', '।', '[SEP]'] ['का', 'Gestalt', 'Gestalt', 1077, 2]
['[CLS]', 'और', 'उसने', 'फिर', 'ब', '##त', '##्ती', 'ब', '##ु', '##झा', 'दी', '।', '[SEP]'] ['उसने', 'Agent', 'Agent', 1079, 2]
['[CLS]', 'फिर', 'उसने', 'एक', 'स', '##फ', '##़', '##ेद', 'और', 'लाल', 'रंग', 'के', 'र', '##ू', '##मा', '##ल', 'से', 'अपनी', 'ल', '##ला', '##ट', 'प', '##ों', '##छ', 'ली', '।', '[SEP]'] ['उसने', 'Agent', 'Agent', 1080, 2]
['[CLS]', 'फिर', 'उसने', 'एक', 'स', '##फ', '##़', '##ेद',

IndexError: list index out of range

In [120]:
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder 
import torch

In [133]:
class SNACSDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.mode = mode
        data = [concatenated_embeddings[x[3]][x[4]][0].tolist() + [x[1], x[2]] for x in data]
        df = pd.DataFrame(data, columns=['e' + str(x) for x in range(3072)] + ['scene', 'function'])
        le = LabelEncoder() # one-hot encoding
        
        if self.mode == 'train':
            df = df.dropna()
            self.inp = df.iloc[:,:-2].values
            self.oup = df.iloc[:,-2:].values.reshape(len(data), 1)
        else:
            self.inp = df.values
            
    def __len__(self):
        return len(self.inp)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            inpt  = torch.Tensor(self.inp[idx])
            oupt  = torch.Tensor(self.oup[idx])
            return { 'inp': inpt,
                     'oup': oupt,
            }
        else:
            inpt = torch.Tensor(self.inp[idx])
            return { 'inp': inpt
            }

In [134]:
real_data = SNACSDataset(data)

IndexError: list index out of range